https://github.com/langchain-ai/rag-from-scratch/blob/main/rag_from_scratch_1_to_4.ipynb

some deviations from the source code because i dont wanna pay for embeddings from openai, or hit openai models. All openAI integration is replaced with ollama.

I also removed langsmith integration. don't think it's needed. just a frontend for LLM debugging which i can achieve with `langchain.debug = True`

In [1]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
# Load Documents
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()


In [3]:
# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [4]:
# Deviates from their notebook due to wanting to use locally run embeddings
from langchain_ollama import OllamaEmbeddings

embed = OllamaEmbeddings(
    model="nomic-embed-text"
)

In [5]:
# Embed
vectorstore = Chroma.from_documents(
    documents=splits, 
    embedding=embed)

In [11]:
# 
retriever = vectorstore.as_retriever(
    search_kwargs={"k": 6}, # How many to retrieve
    search_type='mmr'       # 'similarity' by default
)

In [12]:
# How does this retriever actually work?
[print(i.page_content + '\n') for i in retriever.invoke('What is Task Decomposition?')]

Fig. 1. Overview of a LLM-powered autonomous agent system.
Component One: Planning#
A complicated task usually involves many steps. An agent needs to know what they are and plan ahead.
Task Decomposition#
Chain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.

(3) Task execution: Expert models execute on the specific tasks and log results.
Instruction:

With the input and the inference results, the AI assistant needs to describe the process and results. The previous stages can be formed as - User Input: {{ User Input }}, Task Planning: {{ Tasks }}, Model Selection: {{ Model Assignment }}, Task Execution: {{ Predictions }}. You must first answer 

[None, None, None, None, None, None]

In [13]:
# Langchain hub is basically just version control for prompts. Pull a prompt for RAG.
prompt = hub.pull("rlm/rag-prompt")
prompt

/home/zhiyong/.pyenv/versions/3.10.4/envs/rag/lib/python3.10/site-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [14]:
from langchain_ollama.chat_models import ChatOllama
llm = ChatOllama(model="llama3.2:3b-instruct-q8_0", temperature=0)

In [15]:
# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [16]:
# setting debug to true will allow us to see what is langchain actually creating
import langchain 
langchain.debug = True 

In [17]:
# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [18]:
rag_chain.invoke("What is Task Decomposition?")

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "input": "What is Task Decomposition?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<context,question>] Entering Chain run with input:
{
  "input": "What is Task Decomposition?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<context,question> > chain:RunnableSequence] Entering Chain run with input:
{
  "input": "What is Task Decomposition?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<context,question> > chain:RunnablePassthrough] Entering Chain run with input:
{
  "input": "What is Task Decomposition?"
}
[chain/end] [chain:RunnableSequence > chain:RunnableParallel<context,question> > chain:RunnablePassthrough] s] Exiting Chain run with output:
{
  "output": "What is Task Decomposition?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<context,question> > chain:RunnableSequence > chain:format_docs] Entering Chain run with input:
[inputs]
[ch

'Task Decomposition is a technique used to break down complex tasks into smaller, simpler steps, allowing an agent to plan ahead and utilize more test-time computation. This process transforms big tasks into multiple manageable tasks, enabling the model to shed light on its thinking process. By doing so, Task Decomposition helps to make complex tasks more interpretable and easier to execute.'